# BA Trading System - Code Erklärung Tutorial

Dieses Notebook erklärt den Code des BA Trading Systems Zeile für Zeile.

## Inhaltsverzeichnis

1. [Einführung](#einführung)
2. [ConfigManager.py](#configmanager)
3. [Datagrabber.py](#datagrabber)
4. [Dataprep.py](#dataprep)
5. [Models_Wrapper.py](#models-wrapper)
6. [ModelComparison.py](#modelcomparison)
7. [main.py](#main)
8. [Vollständiges Beispiel](#beispiel)


## 1. Einführung

Das BA Trading System ist ein Portfolio-basiertes Machine Learning System für Aktienprognosen.

### Projektstruktur:
- **ConfigManager.py**: Verwaltet Konfiguration aus YAML
- **Datagrabber.py**: Holt Daten von LSEG/Refinitiv API
- **Dataprep.py**: Feature Engineering und Datenaufbereitung
- **Models_Wrapper.py**: Wrapper für ML-Modelle (PyTorch, Sklearn, etc.)
- **ModelComparison.py**: Orchestriert Training und Vergleich
- **main.py**: Hauptprogramm mit CLI


In [ ]:
# Import notwendiger Bibliotheken
import sys
from pathlib import Path

# Stelle sicher, dass wir im richtigen Verzeichnis sind
sys.path.insert(0, str(Path.cwd()))

print("✓ Imports erfolgreich")


---

## 2. ConfigManager.py - Zeile für Zeile Erklärung

Der ConfigManager lädt und verwaltet die Konfiguration aus `config.yaml`.


In [ ]:
# ZEILE 1-9: Imports
# ====================

import yaml  # Für YAML-Dateien
from pathlib import Path  # Für Pfad-Operationen
from typing import Any, Dict  # Type Hints
from copy import deepcopy  # Für tiefe Kopien
from logger_config import get_logger  # Logging

logger = get_logger(__name__)  # Logger für dieses Modul

print("✓ Imports erklärt")


In [ ]:
# ZEILE 11-80: DEFAULT_CONFIG
# ============================
# Dies ist ein Dictionary mit Standard-Werten, die verwendet werden,
# wenn die config.yaml fehlt oder unvollständig ist.

DEFAULT_CONFIG = {
    "data": {
        "portfolios": {},  # Wird aus config.yaml geladen
        "common_indices": [],  # Gemeinsame Indizes (z.B. VDAX)
        "fields": ["OPEN_PRC", "HIGH_1", "LOW_1", "TRDPRC_1", "ACVOL_1"],
        "periods": {
            "daily": {
                "interval": "daily",
                "start": "2024-01-01",
                "end": "2025-11-15"
            },
            "intraday": {
                "interval": "30min",
                "start": "2024-01-01",
                "end": "2025-11-15"
            }
        }
    },
    # ... weitere Defaults
}

print("✓ DEFAULT_CONFIG erklärt")


In [ ]:
# ZEILE 82-100: __init__ Methode
# ===============================
# Diese Methode wird aufgerufen, wenn ein ConfigManager-Objekt erstellt wird.

from ConfigManager import ConfigManager

# Beispiel: ConfigManager erstellen
config = ConfigManager("config.yaml")

print("✓ ConfigManager initialisiert")
print(f"Config-Pfad: {config.path}")


In [ ]:
# ZEILE 102-130: _load_and_validate_config()
# ===========================================
# Diese Methode:
# 1. Lädt die config.yaml
# 2. Merged sie mit Defaults
# 3. Validiert die Werte

# Beispiel: Config-Werte abrufen
epochs = config.get("models.pytorch_nn.epochs")
print(f"PyTorch Epochs: {epochs}")

# Punkt-Notation: "models.pytorch_nn.epochs" wird zu config["models"]["pytorch_nn"]["epochs"]
features = config.get("features.input_features")
print(f"Features: {features}")


---

## 3. Datagrabber.py - Zeile für Zeile Erklärung

Der DataGrabber holt Daten von der LSEG/Refinitiv API.


In [ ]:
# ZEILE 67-141: fetch_portfolio_data() - DETAILLIERTE ERKLÄRUNG
# =============================================================

print("""
fetch_portfolio_data(portfolio_name, period_type) macht:

1. Lädt Portfolio-Config aus ConfigManager
   - portfolio_name: z.B. "dax" oder "sdax"
   - period_type: "daily" oder "intraday"

2. Konvertiert Datum-Strings zu datetime-Objekten
   start = datetime.datetime.strptime("2024-01-01", "%Y-%m-%d")
   end = datetime.datetime.strptime("2025-11-15", "%Y-%m-%d")

3. Holt Portfolio-Aktien-Daten
   portfolio_df = LS.getHistoryData(
       universe=["RHMG.DE", "ENR1n.DE", ...],  # Aktien
       fields=["TRDPRC_1", "ACVOL_1", ...],    # Felder
       start=start,
       end=end,
       interval="daily"
   )

4. Holt Portfolio-Index (z.B. DAX)
   index_df = LS.getHistoryData(
       universe=[".GDAXI"],
       fields=["TRDPRC_1"],
       ...
   )

5. Holt gemeinsame Indizes (z.B. VDAX)
   common_df = LS.getHistoryData(
       universe=[".V1XI"],
       ...
   )

6. Kombiniert alle DataFrames
   combined_df = pd.concat([portfolio_df, index_df, common_df], axis=1)

7. Speichert als Excel
   exceltextwriter(combined_df, "dax_daily")

8. Gibt DataFrame zurück
""")


---

## 4. Dataprep.py - Zeile für Zeile Erklärung

Dataprep bereitet Daten für Machine Learning vor und erstellt Features.


In [ ]:
# ZEILE 56-233: create_features() - DETAILLIERTE ERKLÄRUNG
# ========================================================

print("""
create_features(df, portfolio_name) - Schritt für Schritt:

SCHRITT 1: DataFrame vorbereiten
===============================
df = df.copy()  # Defensive Kopie (Original bleibt unverändert)

# Stelle sicher, dass Index ein DatetimeIndex ist
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')

SCHRITT 2: Fehlende Werte behandeln
===================================
# Forward Fill (ffill): Fülle mit letztem bekannten Wert
# Backward Fill (bfill): Fülle mit nächstem bekannten Wert
price_columns = [col for col in df.columns if 'TRDPRC_1' in col]
df[price_columns] = df[price_columns].ffill().bfill()

SCHRITT 3: Portfolio-Durchschnittspreis berechnen
=================================================
stock_columns = [col for col in df.columns if '.DE' in col and 'TRDPRC_1' in col]
portfolio_prices = df[stock_columns].mean(axis=1)  # Durchschnitt über alle Aktien

SCHRITT 4: Momentum-Features
===========================
for period in [5, 10, 20]:
    features[f'momentum_{period}'] = portfolio_prices.pct_change(period)
    # pct_change(5) = (price[t] - price[t-5]) / price[t-5]
    # Beispiel: momentum_5[100] = (price[100] - price[95]) / price[95]

SCHRITT 5: Index-Features (DAX/SDAX)
====================================
index_columns = [col for col in df.columns if 'GDAXI' in col]
index_prices = df[index_columns[0]]
features['change_dax'] = index_prices.pct_change()
# Prozentuale Änderung des DAX-Index

SCHRITT 6: VDAX (Volatilität)
=============================
vdax_columns = [col for col in df.columns if 'V1XI' in col]
features['vdax_absolute'] = df[vdax_columns[0]].abs()
# Absoluter Wert des VDAX (Volatilitätsindex)

SCHRITT 7: Volume-Ratio
=======================
volume_columns = [col for col in df.columns if 'VOLUME' in col]
portfolio_volume = df[volume_columns].mean(axis=1)
rolling_mean = portfolio_volume.rolling(20).mean()  # 20-Perioden Durchschnitt
features['volume_ratio'] = portfolio_volume / rolling_mean
# Verhältnis aktuelles Volume zu Durchschnitt

SCHRITT 8: Volatilitäts-Features
=================================
portfolio_returns = np.log(portfolio_prices / portfolio_prices.shift(1))
features['rolling_volatility_10'] = portfolio_returns.rolling(10).std()
# Standardabweichung der Returns über 10 Perioden

SCHRITT 9: Zeit-Features (für Intraday)
========================================
if isinstance(df.index, pd.DatetimeIndex):
    # Wochentag (0=Montag, 6=Sonntag)
    dow = df.index.weekday
    features['dow_sin'] = np.sin(2 * np.pi * dow / 7)  # Zyklische Kodierung
    features['dow_cos'] = np.cos(2 * np.pi * dow / 7)
    
    # Stunde (für Intraday)
    hours = df.index.hour
    features['hour_sin'] = np.sin(2 * np.pi * hours / 24)
    features['hour_cos'] = np.cos(2 * np.pi * hours / 24)

SCHRITT 10: Target-Variable
===========================
features['price_change_next'] = portfolio_returns.shift(-1)
# shift(-1) verschiebt um 1 nach oben
# price_change_next[t] = return[t+1] (zukünftiger Return)

SCHRITT 11: NaN-Werte entfernen
===============================
# Entferne erste N Zeilen (wegen Rolling-Windows)
features = features.iloc[max(momentum_periods):]
# Entferne Zeilen mit NaN im Target
features = features.dropna(subset=['price_change_next'])

Gibt DataFrame mit Features zurück
""")


In [ ]:
# ZEILE 292-344: time_series_split() - WICHTIG!
# =============================================
# Chronologischer Train-Test Split (KEIN Random Shuffle!)

print("""
time_series_split(X, y, test_size=0.2) macht:

1. Berechne Split-Index
   n_samples = len(X)  # z.B. 1000
   split_idx = int(n_samples * (1 - test_size))  # z.B. 800
   # Train: 0-800, Test: 800-1000

2. Validiere Größen
   if n_train < 50:
       raise ValueError("Trainingsset zu klein")
   if n_test < 10:
       raise ValueError("Testset zu klein")

3. Split (chronologisch!)
   X_train = X.iloc[:split_idx]  # Erste 80%
   X_test = X.iloc[split_idx:]   # Letzte 20%
   y_train = y.iloc[:split_idx]
   y_test = y.iloc[split_idx:]

WARUM chronologisch?
===================
Bei Zeitreihen müssen wir die Zeitordnung erhalten:
- Training: Vergangene Daten (z.B. 2024-01-01 bis 2024-12-31)
- Test: Zukünftige Daten (z.B. 2025-01-01 bis 2025-11-15)

Random Shuffle würde Data Leakage verursachen!
""")


---

## 5. Models_Wrapper.py - Zeile für Zeile Erklärung

Models_Wrapper enthält Wrapper-Funktionen für alle ML-Modelle.


In [ ]:
# ZEILE 74-252: train_pytorch_model() - DETAILLIERT
# ==================================================

print("""
train_pytorch_model() - Schritt für Schritt:

SCHRITT 1: Seeds setzen (Reproduzierbarkeit)
============================================
np.random.seed(42)
torch.manual_seed(42)
# Gleiche Seeds = gleiche Ergebnisse bei jedem Lauf

SCHRITT 2: Device wählen (CPU oder GPU)
=======================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# GPU ist schneller, aber nicht immer verfügbar

SCHRITT 3: Daten zu Tensoren konvertieren
==========================================
X_train_t = torch.tensor(X_train.values, dtype=torch.float32)
y_train_t = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)
# PyTorch braucht Tensoren, nicht DataFrames

SCHRITT 4: Validation Split
===========================
n_train = len(X_train_t)  # z.B. 800
val_idx = int(n_train * (1 - validation_split))  # z.B. 640
X_train_inner = X_train_t[:val_idx]  # 0-640: Training
X_val = X_train_t[val_idx:]          # 640-800: Validation

SCHRITT 5: Target standardisieren (optional)
============================================
y_mean = y_train_inner.mean()
y_std = y_train_inner.std()
y_train_std = (y_train_inner - y_mean) / y_std
# Standardisierung hilft beim Training (bessere Konvergenz)

SCHRITT 6: Modell erstellen
===========================
model = SimpleNet(in_features=n_features, hidden1=128, hidden2=64)
model = model.to(device)  # Auf GPU verschieben falls verfügbar

SCHRITT 7: Optimizer und Loss
=============================
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
criterion = nn.MSELoss()  # Mean Squared Error

SCHRITT 8: Training Loop
========================
for epoch in range(epochs):  # z.B. 400 Epochen
    # Training
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()        # Gradienten zurücksetzen
        outputs = model(batch_X)     # Forward Pass
        loss = criterion(outputs, batch_y)  # Loss berechnen
        loss.backward()              # Backward Pass (Gradienten)
        optimizer.step()             # Gewichte updaten
    
    # Validation
    with torch.no_grad():  # Keine Gradienten für Validation
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val_std)
    
    # Early Stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_model_state = model.state_dict()  # Speichere beste Gewichte
    else:
        patience_counter += 1
        if patience_counter >= patience:
            break  # Stoppe Training

SCHRITT 9: Bestes Modell laden
==============================
model.load_state_dict(best_model_state)  # Lade beste Gewichte

SCHRITT 10: Predictions und Metriken
====================================
y_pred = model(X_test)
y_pred = y_pred * y_std + y_mean  # Zurück in Original-Skala
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

Gibt (model, metrics) zurück
""")


---

## 6. ModelComparison.py - Zeile für Zeile Erklärung

ModelComparison orchestriert den kompletten Workflow.


In [ ]:
# ZEILE 41-110: run_full_comparison() - HAUPT-WORKFLOW
# =====================================================

print("""
run_full_comparison() - Haupt-Workflow:

SCHRITT 1: Daten holen
======================
grabber = DataGrabber(config_path)
all_data = grabber.fetch_all_data()
# Returns: {"dax": {"daily": df, "intraday": df}, "sdax": {...}}

SCHRITT 2: Für jedes Portfolio und jede Periode
================================================
for portfolio_name, portfolio_data in all_data.items():
    for period_type, data in portfolio_data.items():
        # z.B. portfolio_name="dax", period_type="daily"
        
        # Datenaufbereitung
        prep = DataPrep(config_path)
        X, y = prep.prepare_data(data, portfolio_name, period_type)
        
        # Train-Test Split
        X_train, X_test, y_train, y_test = time_series_split(X, y, test_size=0.2)
        
        # SKALIERUNG (wichtig!)
        scaler = StandardScaler()
        scaler.fit(X_train)  # Fit NUR auf Training!
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        # Warum nur auf Training fit? -> Verhindert Data Leakage!
        
        # Modelle trainieren
        results = train_all_models(X_train, X_test, y_train, y_test, ...)
        
        # Ergebnisse speichern
        self.results[f"{portfolio_name}_{period_type}"] = results

SCHRITT 3: Vergleichsbericht erstellen
======================================
create_comparison_report()
# Erstellt Excel mit allen Metriken
""")


---

## 7. Vollständiges Beispiel - Wie alles zusammenarbeitet


In [ ]:
# VOLLSTÄNDIGES BEISPIEL
# ======================

print("""
WORKFLOW - Schritt für Schritt:

1. USER startet Programm:
   $ python main.py --mode daily --models pytorch_nn random_forest

2. main.py:
   - Lädt config.yaml
   - Überschreibt mit CLI-Argumenten
   - Erstellt ModelComparison-Objekt

3. ModelComparison.run_full_comparison():
   a) DataGrabber.fetch_all_data():
      - Holt Daten für DAX und SDAX
      - Für Daily und Intraday
      - Speichert als Excel
   
   b) Für jedes Portfolio/Periode:
      - DataPrep.prepare_data():
        * Erstellt Features (Momentum, Index, etc.)
        * Erstellt X und y
      - time_series_split():
        * Train: 80%, Test: 20%
      - Skalierung:
        * StandardScaler auf X_train fitten
        * Auf X_train und X_test anwenden
      - train_all_models():
        * Trainiert PyTorch NN
        * Trainiert Random Forest
        * Berechnet Metriken
   
   c) create_comparison_report():
      - Sammelt alle Metriken
      - Erstellt Excel-Report
      - Zeigt beste Modelle

4. ERGEBNIS:
   - Results/model_comparison.xlsx
   - Models/dax_daily/pytorch_nn.pt
   - Models/dax_daily/random_forest.pkl
   - Logs/main.log

DATENFLUSS:
===========
config.yaml
    ↓
ConfigManager
    ↓
DataGrabber → DataFrame (Rohdaten)
    ↓
DataPrep → (X, y) (Features + Target)
    ↓
time_series_split → (X_train, X_test, y_train, y_test)
    ↓
Scaler → (X_train_scaled, X_test_scaled)
    ↓
Models_Wrapper → (model, metrics)
    ↓
ModelComparison → Excel-Report
""")


---

## Wichtige Konzepte

### 1. Data Leakage Prävention

- **Skalierung**: Scaler wird NUR auf X_train gefittet
- **Time Series Split**: Chronologisch, kein Random Shuffle
- **Features**: Nur vergangene Informationen

### 2. Portfolio-basiertes Training

- Ein Modell pro Portfolio (nicht pro Aktie)
- Y = Durchschnittliche Return aller Aktien
- Robustere Predictions

### 3. Feature Engineering

- Momentum: Prozentuale Änderung über N Perioden
- Index-Änderung: DAX/SDAX Return
- Volatilität: Rolling Standard Deviation
- Zeit-Features: Zyklische Kodierung (Sinus/Cosinus)

### 4. Modell-Vergleich

- Alle Modelle werden auf gleichen Daten trainiert
- Metriken: R², MSE, MAE, Directional Accuracy
- Excel-Report für einfachen Vergleich

---

## Zusammenfassung

Dieses Tutorial hat den Code Zeile für Zeile erklärt:

1. **ConfigManager**: Lädt und validiert Konfiguration
2. **DataGrabber**: Holt Daten von API
3. **DataPrep**: Feature Engineering und Datenaufbereitung
4. **Models_Wrapper**: ML-Modell Training
5. **ModelComparison**: Orchestriert Workflow
6. **main.py**: Einstiegspunkt

**Nächste Schritte:**
- Experimentiere mit verschiedenen Features
- Passe Hyperparameter an
- Füge neue Modelle hinzu

Viel Erfolg! 🚀


---

## Directional Accuracy - Detaillierte Erklärung

**Directional Accuracy** ist eine wichtige Metrik für Trading-Systeme!


In [ ]:
# DIRECTIONAL ACCURACY - Was bedeutet das?

import numpy as np
import pandas as pd

print("""
WAS IST DIRECTIONAL ACCURACY?
============================

Directional Accuracy misst, wie oft das Modell die RICHTUNG der Preisänderung
korrekt vorhersagt - also ob der Preis STEIGT (+) oder FÄLLT (-).

WARUM WICHTIG?
=============
Bei Aktienprognosen ist es oft wichtiger, die RICHTUNG korrekt vorherzusagen
als den exakten Wert. Für Trading-Entscheidungen (Kaufen/Verkaufen) ist die
Richtung das Entscheidende!
""")


In [ ]:
# BEISPIEL-BERECHNUNG

# Beispiel: Tatsächliche Returns
y_true = np.array([0.02, -0.01, 0.005, -0.015, 0.01, -0.005, 0.03, -0.02])

# Beispiel: Vorhergesagte Returns
y_pred = np.array([0.018, -0.008, -0.002, -0.012, 0.009, -0.003, 0.025, -0.018])

# Schritt 1: Vorzeichen bestimmen
true_signs = np.sign(y_true)  # +1 für positiv, -1 für negativ
pred_signs = np.sign(y_pred)

print("Tatsächliche Returns:", y_true)
print("Tatsächliche Vorzeichen:", true_signs)
print("\nVorhergesagte Returns:", y_pred)
print("Vorhergesagte Vorzeichen:", pred_signs)

# Schritt 2: Vergleich
correct = (true_signs == pred_signs)

print("\nKorrekt?", correct)

# Schritt 3: Anteil berechnen
accuracy = np.mean(correct)

print(f"\n✓ Directional Accuracy: {accuracy:.1%}")
print(f"  ({np.sum(correct)} von {len(y_true)} Vorhersagen korrekt)")


In [ ]:
# INTERPRETATION DER WERTE

print("""
WIE INTERPRETIERE ICH DIRECTIONAL ACCURACY?

50% (0.50)  → Zufällig (Münzwurf)
             → Modell ist nutzlos

55-60%      → Leicht besser als Zufall
             → Modell hat etwas gelernt
             → Für Finanzdaten: GUT!

60-65%      → Gut
             → Modell kann Richtung gut vorhersagen
             → Potentiell profitabel für Trading

>65%        → Sehr gut
             → Selten bei Finanzdaten!

WICHTIG:
========
Bei Finanzdaten ist bereits 55-60% Directional Accuracy SEHR GUT!
Selbst professionelle Trader haben oft nur 52-55% Win-Rate.

Deine Werte von 58-62% in der Excel sind also EXCELLENT! 🎯
""")


In [ ]:
# PRAKTISCHES BEISPIEL: Trading-Strategie

print("""
WIE KANN ICH DIRECTIONAL ACCURACY NUTZEN?

Beispiel-Strategie mit 62% Directional Accuracy:

1. Modell sagt "Preis STEIGT" voraus (↑)
   → KAUFE Aktie
   → 62% Chance auf Gewinn

2. Modell sagt "Preis FÄLLT" voraus (↓)
   → VERKAUFE Aktie (oder Short)
   → 62% Chance auf Gewinn

Von 100 Trades:
- 62 korrekte Vorhersagen → Gewinn
- 38 falsche Vorhersagen → Verlust

Wenn durchschnittlicher Gewinn > durchschnittlicher Verlust:
→ STRATEGIE IST PROFITABEL! 💰

WARUM WICHTIGER ALS R²?
=======================
R² = 0.15 (niedrig) bedeutet:
- Exakte Werte schwer vorherzusagen
- ABER: Directional Accuracy = 0.62 (gut!) bedeutet:
- Richtung kann gut vorhergesagt werden
- Für Trading-Entscheidungen ist das oft ausreichend!
""")


In [ ]:
# CODE-ZEILE FÜR ZEILE ERKLÄRUNG

print("""
def directional_accuracy(y_true, y_pred):
    \"\"\"Berechnet Trefferrate der Vorzeichen.\"\"\"
    
    # ZEILE 54-55: Prüfe ob Daten vorhanden
    if len(y_true) == 0:
        return np.nan  # Keine Daten = NaN
    
    # ZEILE 56-57: Konvertiere zu Arrays und flatten
    y_true = np.asarray(y_true).flatten()
    y_pred = np.asarray(y_pred).flatten()
    # .flatten() macht sicher, dass es 1D-Array ist
    
    # ZEILE 58: Berechne Directional Accuracy
    return float(np.mean(np.sign(y_true) == np.sign(y_pred)))
    
    # Schritt für Schritt:
    # 1. np.sign(y_true) → [+1, -1, +1, ...]  (Vorzeichen)
    # 2. np.sign(y_pred) → [+1, -1, -1, ...]  (Vorzeichen)
    # 3. == → [True, True, False, ...]  (Vergleich)
    # 4. np.mean() → 0.75  (Anteil True-Werte)
    # 5. float() → 0.75  (konvertiert zu Float)
""")


In [ ]:
# VISUELLES BEISPIEL

import pandas as pd

# Erstelle Beispiel-Datenframe
df_example = pd.DataFrame({
    'Tatsächlicher_Return': [0.02, -0.01, 0.005, -0.015, 0.01, -0.005],
    'Vorhergesagter_Return': [0.018, -0.008, -0.002, -0.012, 0.009, -0.003],
})

# Berechne Vorzeichen
df_example['True_Sign'] = np.sign(df_example['Tatsächlicher_Return'])
df_example['Pred_Sign'] = np.sign(df_example['Vorhergesagter_Return'])
df_example['Korrekt'] = df_example['True_Sign'] == df_example['Pred_Sign']
df_example['Richtung_True'] = df_example['True_Sign'].map({1: '↑ STEIGT', -1: '↓ FÄLLT', 0: '→ UNVERÄNDERT'})
df_example['Richtung_Pred'] = df_example['Pred_Sign'].map({1: '↑ STEIGT', -1: '↓ FÄLLT', 0: '→ UNVERÄNDERT'})

print("BEISPIEL-BERECHNUNG:")
print("="*80)
print(df_example.to_string(index=False))

accuracy = df_example['Korrekt'].mean()
print(f"\n✓ Directional Accuracy: {accuracy:.1%} ({df_example['Korrekt'].sum()}/{len(df_example)} korrekt)")


### Was bedeuten die Werte in deiner Excel?

**Typische Werte für Finanzdaten:**
- **50%**: Zufällig (wie Münzwurf) - Modell nutzlos
- **55-60%**: **Sehr gut** - Modell kann Richtung gut vorhersagen
- **>60%**: **Exzellent** - Selten bei Finanzdaten
- **>70%**: **Außergewöhnlich** - Extrem selten

**Deine Werte von 58-62% sind sehr gut für Finanzdaten!**

Siehe auch: `DIRECTIONAL_ACCURACY_ERKLAERUNG.md` für detaillierte Erklärung.
